In [1]:

import numpy as np
import pandas as pd
from keras import layers
from keras.layers import Input,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,Dropout,GlobalMaxPooling2D,GlobalAveragePooling2D
from keras.utils.np_utils import to_categorical
from keras.models import Sequential 
from keras.callbacks import ModelCheckpoint
import keras.backend as K
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
#importing the csv file
data=pd.read_csv("data.csv")
dataset=np.array(data)
np.random.shuffle(dataset)
X = data.values[:,:-1]/255.0
Y = data["character"].values


#splitting into train and test
n_classes = 46
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)


In [3]:
print("number of training samples = "+str(x_train.shape[0]))
print("number of test samples = "+str(x_test.shape[0]))
print("X_train shape:"+str(x_train.shape))
print("X_test shape:"+str(x_test.shape))
print("Y_train shape:"+str(y_train.shape))
print("Y_test shape:"+str(y_test.shape))


number of training samples = 64400
number of test samples = 27600
X_train shape:(64400, 1024)
X_test shape:(27600, 1024)
Y_train shape:(64400, 46)
Y_test shape:(27600, 46)


In [4]:
image_x=32
image_y=32
im_shape = (image_x, image_y, 1)
train_y=y_train.reshape(y_train.shape[0],y_train.shape[1])
test_y=y_test.reshape(y_test.shape[0],y_test.shape[1])
x_train=x_train.reshape(x_train.shape[0],*im_shape)
x_test=x_test.reshape(x_test.shape[0],*im_shape)
print("X_train shape:"+str(x_train.shape))
print("Y_train shape:"+str(train_y.shape))
x_train= np.asarray(x_train).astype(np.float32)
x_test=np.asarray(x_test).astype(np.float32)
train_y=np.asarray(train_y).astype(np.float32)
test_y=np.asarray(test_y).astype(np.float32)



X_train shape:(64400, 32, 32, 1)
Y_train shape:(64400, 46)


In [5]:
def keras_model(image_x,image_y):
    model=Sequential()
    model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=im_shape,activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))
    model.add(Conv2D(64,(3,3),activation="relu"))
    model.add(MaxPooling2D(pool_size=(5,5),strides=(5,5),padding='same'))
    model.add(Flatten())
    model.add(Dense(n_classes,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    filepath="devanagri.h5"
    checkpoint1=ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True,mode='max')
    callbacks_list=[checkpoint1]
    
    return model,callbacks_list

In [6]:
#fitting the model
model,callbacks_list=keras_model(image_x,image_y)
model.fit(x_train,train_y,validation_data=(x_test,test_y),epochs=8,batch_size=64,callbacks=callbacks_list)
scores=model.evaluate(x_test,test_y,verbose=0)
print("CNN error:%2f%%"%(100-scores[1]*100))
model.summary()
model.save('devanagri.h5')


Epoch 1/8
1007/1007 [==============================] - 178s 177ms/step - loss: 0.8170 - accuracy: 0.7752 - val_loss: 0.3265 - val_accuracy: 0.9028
Epoch 2/8
1007/1007 [==============================] - 205s 203ms/step - loss: 0.2472 - accuracy: 0.9250 - val_loss: 0.2098 - val_accuracy: 0.9364
Epoch 3/8
1007/1007 [==============================] - 210s 208ms/step - loss: 0.1635 - accuracy: 0.9505 - val_loss: 0.1707 - val_accuracy: 0.9491
Epoch 4/8
1007/1007 [==============================] - 176s 175ms/step - loss: 0.1220 - accuracy: 0.9635 - val_loss: 0.1584 - val_accuracy: 0.9512
Epoch 5/8
1007/1007 [==============================] - 176s 174ms/step - loss: 0.0960 - accuracy: 0.9702 - val_loss: 0.1354 - val_accuracy: 0.9586
Epoch 6/8
1007/1007 [==============================] - 174s 173ms/step - loss: 0.0787 - accuracy: 0.9760 - val_loss: 0.1378 - val_accuracy: 0.9584
Epoch 7/8
1007/1007 [==============================] - 172s 170ms/step - loss: 0.0625 - accuracy: 0.9805 - val_loss: 0